In [ ]:
# %pip install langchain
# %pip install langchain-community
# %pip install pypdf
# %pip install unstructured
# %pip install openpyxl
# %pip install --upgrade --quiet  gpt4all > /dev/null
# %pip install --upgrade --quiet langchain-elasticsearch langchain-openai tiktoken langchain
# %pip install python-dotenv

## Loading Documents

In [ ]:
import os
import pandas as pd
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.docstore.document import Document
import csv


In [ ]:
def convert_excel_to_csv(directory):
    # Get a list of all Excel files in the directory
    excel_files = [file for file in os.listdir(directory) if file.endswith(('.xls', '.xlsx'))]
    for excel_file in excel_files:
        # Define the path to the Excel file
        excel_file_path = os.path.join(directory, excel_file)
        
        # Read all sheets from the Excel file
        excel_data = pd.read_excel(excel_file_path, sheet_name=None)

        # Process each sheet separately
        for sheet_name, df in excel_data.items():
            # Define the output CSV file path, including the sheet name
            output_csv_file = os.path.join(directory, f"{os.path.splitext(excel_file)[0]}_{sheet_name}.csv")

            # Drop unnamed columns
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            
            # Drop rows where all values are NaN or columns that are entirely NaN
            df.dropna(how='all', inplace=True)
            df.dropna(axis=1, how='all', inplace=True)
            
            # Save the DataFrame to a CSV file
            df.to_csv(output_csv_file, index=False, encoding='utf-8')
            
            print(f"Converted {excel_file} - Sheet: {sheet_name} to {output_csv_file}")

In [ ]:
directory = "/home/ethel/Documents/SOSE 2024/DATA SCIENCE/project/DataScienceGroup13/src/KnowledgeBase"
convert_excel_to_csv(directory)

In [ ]:
allDocs = {}

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        print("Processing", filename)
        pdfLoader = PyPDFLoader(os.path.join(directory, filename))

        if "pdfs" not in allDocs:
            allDocs["pdfs"] = []

        allDocs["pdfs"].extend(pdfLoader.load())

    # elif filename.endswith(".xlsx"):
    #     print("Processing", filename)
    #     excelLoader = UnstructuredExcelLoader(os.path.join(directory, filename))


    #     if "xlsx" not in allDocs:
    #         allDocs["xlsx"] = []

    #     allDocs["xlsx"].extend(excelLoader.load())

    # elif filename.endswith(".csv"):
    #     print("Processing", filename)
    #     csvloader = CSVLoader(os.path.join(directory, filename))

    #     if "csv" not in allDocs:
    #         allDocs["csv"] = []

    #     allDocs["csv"].extend(csvloader.load())
    else:
        continue

In [ ]:
# To convert csv file to Langchain doc
def process_csv(file_path):
    docs = []
    with open(file_path, newline="", encoding='utf-8-sig') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        columns = csv_reader.fieldnames  # Get the column names dynamically
        for i, row in enumerate(csv_reader):
            to_metadata = {col: row[col] for col in columns if col in row}
            values_to_embed = {k: row[k] for k in columns if k in row}
            to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
            newDoc = Document(page_content=to_embed, metadata=to_metadata)
            docs.append(newDoc)
    return docs
allDocs['csv'] = []
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        allDocs['csv'].extend(process_csv(file_path))

In [ ]:
print("Loaded", len(allDocs["pdfs"]), "pdf pages")
# print("Loaded", len(allDocs["xlsx"]), "excel sheets")
print("Loaded", len(allDocs["csv"]), "CSV rows")


In [ ]:
# print("First excel page content: \n", allDocs["xlsx"][0].page_content[0:1000])
print("CSV row: \n", allDocs["csv"][400].page_content)

In [ ]:
print("First pdf page content: \n", allDocs["pdfs"][0].page_content[0:1000])

In [ ]:
# print("First excel page metadata:", allDocs["csv"][0].metadata)

In [ ]:
print("First pdf page metadata:", allDocs["pdfs"][78].metadata)

## Splitting Documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

# r_splitter_excel = RecursiveCharacterTextSplitter(
#     chunk_size=150,
#     chunk_overlap=0,
#     separators=["\r\n", "\n", "\t", ",", " "]
# )
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500, 
    chunk_overlap=0,
    length_function=len
    )


In [ ]:
splits={}

for key in allDocs:
    splits[key] = r_splitter.split_documents(allDocs[key])

In [ ]:
for key in allDocs:
    print("Number of splits for", key, ":", len(splits[key]))

In [ ]:
print("First 5 PDF splits: ", splits["pdfs"][:5])

In [ ]:
# print("First 5 Excel splits", splits["xlsx"][:5])
# print("First 5 CSV splits", splits["csv"][:5])
splits["csv"][0].page_content


## Create Embeddings

In [ ]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_elasticsearch import ElasticsearchStore
import numpy as np
from dotenv import load_dotenv
import os

In [ ]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"

In [ ]:
embedding = GPT4AllEmbeddings(model_name=model_name)

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
load_dotenv()  # This loads the .env file at the application start
password = os.getenv('passwd')
api_key = os.getenv('api_key')

In [ ]:
cloud_id  = '802f868877384e9798b731802ffa4827:ZXVyb3BlLXdlc3QzLmdjcC5jbG91ZC5lcy5pbyQ0NzYyZTQ2YzQ5NDg0ODY5YTAzZDMxYzg5NjY2MjY3YyQ1ZjQ3NWI2NTQxOTI0NmZiODcxNDc3NjZlMTI4YWE2YQ=='
elastic_vector_search = ElasticsearchStore(
    es_cloud_id=cloud_id,
    index_name="embeddings_index",
    embedding=embedding,
    es_user="group13",
    es_password=password,
    es_api_key=api_key
)

In [ ]:
# elastic_vector_search.delete("embeddings_index")

In [ ]:
elastic_vector_search.add_documents(splits["pdfs"])

In [ ]:
elastic_vector_search.add_documents(splits["csv"])

## Testing

In [ ]:
question = "Who is responsible for conducting the risk analysis and what methodology is used"

results = elastic_vector_search.similarity_search(question, 20)

In [ ]:
len(results)

In [ ]:
results[0].page_content